# Searching $n\times n$ grids for optimal population distribution. 

We want to first import the necessary packages: DelimitedFiles for importing our comma-separated txt with the list of all districting plans, as well as ProgressBars so we can see the progress of the final run. 

In [ ]:
# using Pkg
# Pkg.add("DelimitedFiles")
# Pkg.add("ProgressBars")
using DelimitedFiles
using ProgressBars

We set our constants, dim dimension of the tile and n num of districting plans, this allows us to generate our districting plan lookup table easily. [The value $n$ is given by research from MGGG. ](https://mggg.org/table.html)

In [ ]:
dim = 5
n = 4006

## Loading all plans
`load_plan(loc)` loads the $n$ plans from a `.txt` file named `loc`. This file is generated by code from [Zachary Schutzman](https://github.com/zschutzman/enumerator). It loads the possible districting plans into an $dim\times dim\times n$ array of tuples (representing coordinates). 

In [ ]:
function load_plan(loc)
    a = Array{Tuple{UInt8,UInt8}}(undef, dim, dim, n)
    f = readdlm(loc, ',', Int, '\n')
    planCount = 1
    for plan in eachrow(f)
        idx = 1
        dist_count = [0, 0, 0, 0, 0]
        for r in collect(1:dim)
            for c in collect(1:dim)
                dist = plan[idx]
                dist_count[dist] += 1
                a[dist_count[dist],plan[idx],planCount] = (r,c)
                idx += 1
            end
        end
        planCount += 1
    end
    return a
end

plans = load_plan("five.txt")

## Test a given population with a plan
`test_population(delta, plan)` tests a given population `delta` on a districting plan `plan`. `delta` and `plan` expect matching square arrays. `plan` should contain $dim$ rows of $dim$ coordinates (tuples) each, and `delta` is preferably a bit array to optimize memory. 

In [ ]:
vote_threshold = 2
function test_population(delta, plan)
    seats = 0
    for n in eachcol(plan)
        district_votes = 0
        for coord in n
            district_votes += delta[coord[1],coord[2]]
        end
        if district_votes > vote_threshold
                seats += 1
        end
    end
    return seats
end

## Clustering and counts

`clus(delta)` checks for the degree of clustering of a given population. It checks for $P(\Delta(p_i) = \Delta(p_j))$, where  $p_i$, $p_j$ are adjacent and $\Delta(p_k)$ is the voter affiliation of $p_k$. `clus` returns a tuple, one is the clustering for $1$ and the other is clustering for all. Note that `clus(delta)` relies on bitwise operators, so `delta` is (again) preferably a bit array. 

`numH(delta)` simply enumerates the number of $1$s in the grid. We treat a $1$ as a voter for the $\heartsuit$ party. 

In [ ]:
function clus(delta)
    total_con = [0, 0]
    con = [0, 0]
    for r in collect(1:dim)
        for c in collect(1:dim)
            for (dx, dy) in [(1, 0), (-1, 0), (0, -1), (0, 1)]
                nr = r + dx
                nc = c + dy
                if 1 <= nr <= dim && 1 <= nc <= dim
                    total_con[delta[r,c] + 1] += 1
                    con[delta[r,c] + 1] += ~(delta[r,c] ⊻ delta[nr,nc])
                end
            end
        end
    end
    return ((con[1] + con[2])/(total_con[1] + total_con[2]), con[2]/total_con[2])
end

function numH(delta)
    n = 0
    for r in delta
        n += r
    end
    return n
end

## Testing population over all plans

`test_delta(delta, plans)` does `test_population` on all plans in a plan array `plans`. 

## Iterating through all populations

The program iterates through all possibilities of `delta` by iterating through the first $2^{dim\times dim}$ integers, generating all possibilities through the bitwise splitting. `to_grid` generates the `BitArray` grid from a number $0\leq n \leq 2^{25}$. 

In [ ]:
function test_delta(delta, plans)
    result = [0, 0, 0, 0, 0, 0]
    for plan_num in collect(1:size(plans)[3])
        result[test_population(delta, plans[:,:,plan_num]) + 1] += 1
    end
    return result
end

function to_grid(n)
    b = BitArray(undef, 25)
    digits!(b, n, base = 2)
    delta = BitArray(undef, 5, 5)
    delta[1,:] = b[1:5]
    delta[2,:] = b[6:10]
    delta[3,:] = b[11:15]
    delta[4,:] = b[16:20]
    delta[5,:] = b[21:25]
    return delta
end

## Running the simulation
`simulate(filename, times` simulates `times` number of `delta` over **all** `plans`. It outputs the result to a `csv` formatted file `filename`. Setting `times` to $2^{25}-1$ gives all plans. 

In [ ]:
function simulate(filename, times)
    io = open(filename, "w")
    write(io, "Clus,ClusH,NumH,0,1,2,3,4,5,N\n")
    for n in ProgressBar(collect(0:times))
        b = BitArray(undef, 25)
        digits!(b, n, base = 2)
        delta = BitArray(undef, 5, 5)
        delta[1,:] = b[1:5]
        delta[2,:] = b[6:10]
        delta[3,:] = b[11:15]
        delta[4,:] = b[16:20]
        delta[5,:] = b[21:25]
        cluster_score = clus(delta)
        test = test_delta(delta, plans)
        line = "$(cluster_score[1]),$(cluster_score[2]),$(numH(delta)),$(test[1]),$(test[2]),$(test[3]),$(test[4]),$(test[5]),$(test[6]),$n\n"
        write(io, line)
    end
    close(io)
end

You may run the simulation from this jupyter notebook, but exporting it as an executable and running it from the command line is advised for simulations of larger magnitude (do File > Download As > Julia (.jl)). This script is really only feasible for $dim \leq 5$, as it runs in exponential time $O(2^{p(n)})$. 

In [ ]:
simulate("five.csv", 2 ^ 25 - 1)